In [1]:
from AfricaBorderConflictRegression_functions import *

In [2]:
grid_10km = import_geopackage("../Data/Regression_data/Africa-border-conflict_regression-data_102022.gpkg", "grid_10km_102022")
grid_25km = import_geopackage("../Data/Regression_data/Africa-border-conflict_regression-data_102022.gpkg", "grid_25km_102022")
grid_50km = import_geopackage("../Data/Regression_data/Africa-border-conflict_regression-data_102022.gpkg", "grid_50km_102022")
grid_100km = import_geopackage("../Data/Regression_data/Africa-border-conflict_regression-data_102022.gpkg", "grid_100km_102022")

# 3.1 Regression Analysis

### OLS Regression - no controls

In [3]:
grid = grid_50km
dist_cols = ["dist_national_borders_102023_km","dist_colonial_borders_102023_km","dist_religious_borders_102023_km","dist_ethnic_borders_102023_km",]
y_col = "count_total_aggregated"
models = {}
for d in dist_cols:
    df = grid[[d, y_col]].copy()
    df = df.replace([np.inf, -np.inf], np.nan).dropna()
    df["km_closer"] = (-df[d].astype(float)) / 100.0   # <-- now per 100 km closer
    y = df[y_col].astype(float)
    X = sm.add_constant(df["km_closer"])
    m = sm.OLS(y, X).fit(cov_type="HC1")
    models[d] = m
    print("\n" + "="*95)
    print(f"OLS Regression: increase in conflict count per 100km closer to border) | {d}")
    print("="*95)
    print(m.summary().tables[1])


OLS Regression: increase in conflict count per 100km closer to border) | dist_national_borders_102023_km
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         17.8474      1.122     15.901      0.000      15.647      20.047
km_closer      0.9245      0.773      1.195      0.232      -0.592       2.441

OLS Regression: increase in conflict count per 100km closer to border) | dist_colonial_borders_102023_km
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         20.6095      1.098     18.774      0.000      18.458      22.761
km_closer      1.6756      0.315      5.316      0.000       1.058       2.293

OLS Regression: increase in conflict count per 100km closer to border) | dist_religious_borders_102023_km
                 coef    std err          z      

### OLS with controls

In [4]:
grid = grid_50km
dist_cols = ["dist_national_borders_102023_km","dist_colonial_borders_102023_km","dist_religious_borders_102023_km","dist_ethnic_borders_102023_km"]
y_col = "count_total_aggregated"
models = {}
for d in dist_cols:
    cols = [d, y_col, "is_coastal", "pop_est", "road_km", "landcover_majority", "lisa_label"]
    df = grid[cols].copy()
    df = df.replace([np.inf, -np.inf], np.nan).dropna()
    df["km_closer"] = (-df[d].astype(float)) / 100.0
    df["log_pop"] = np.log(df["pop_est"].astype(float).clip(lower=1))
    df["log_road"] = np.log(df["road_km"].astype(float).clip(lower=1e-6))
    df["landcover_majority"] = df["landcover_majority"].astype("category")
    df["lisa_label"] = df["lisa_label"].astype("category")
    formula = f"""{y_col} ~ km_closer + is_coastal + log_pop + log_road + C(landcover_majority, Treatment(reference=11)) + C(lisa_label, Treatment(reference="Not signif"))"""
    m = smf.ols(formula, data=df).fit(cov_type="HC1")
    models[d] = m
    print(f"OLS with controls | increase in conflict count per 100km closer to border | {d}")
    print(m.summary().tables[1])

OLS with controls | increase in conflict count per 100km closer to border | dist_national_borders_102023_km
                                                             coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                  0.9086      1.152      0.788      0.430      -1.350       3.167
C(landcover_majority, Treatment(reference=11))[T.1.0]      0.2523      7.489      0.034      0.973     -14.426      14.931
C(landcover_majority, Treatment(reference=11))[T.2.0]      1.9078      1.744      1.094      0.274      -1.510       5.326
C(landcover_majority, Treatment(reference=11))[T.4.0]    -11.2209      6.013     -1.866      0.062     -23.006       0.564
C(landcover_majority, Treatment(reference=11))[T.5.0]     16.3109      7.714      2.114      0.034       1.192      31.430
C(landcover_majority, Treatment

### Statistically significant & positive coefficents for proximity to border

In [5]:
grid = grid_50km
dist_cols = ["dist_national_borders_102023_km","dist_colonial_borders_102023_km","dist_religious_borders_102023_km","dist_ethnic_borders_102023_km"]
count_cols = ["count_total_aggregated","count_battles", "count_explosions_remote_violence", "count_protests","count_riots", "count_strategic_developments","count_violence_against_civilians", "count_total_disaggregated","count_abduction_forced_disappearance", "count_agreement","count_air_drone_strike", "count_armed_clash", "count_arrests","count_attack", "count_change_to_group_activity","count_chemical_weapon", "count_disrupted_weapons_use","count_excessive_force_against_protesters","count_government_regains_territory", "count_grenade","count_headquarters_or_base_established","count_looting_property_destruction", "count_mob_violence","count_non_state_actor_overtakes_territory","count_non_violent_transfer_of_territory", "count_other","count_peaceful_protest", "count_protest_with_intervention","count_remote_explosive_landmine_ied", "count_sexual_violence", "count_shelling_artillery_missile_attack", "count_suicide_bomb","count_violent_demonstration", "count_total_subevent","fatalities_total",]
controls = ["is_coastal", "pop_est", "road_km", "landcover_majority", "lisa_label"]
alpha = 0.01
results_by_border = {}
sig_by_border = {}

for d in dist_cols:
    rows = []
    for y_col in count_cols:
        cols = [d, y_col] + controls
        df = grid[cols].copy().replace([np.inf, -np.inf], np.nan).dropna()
        if df.empty or df[y_col].nunique() < 2 or df[d].nunique() < 2:
            continue
        df["km_closer"] = (-df[d].astype(float)) / 100.0
        df["log_pop"]  = np.log(df["pop_est"].astype(float).clip(lower=1))
        df["log_road"] = np.log(df["road_km"].astype(float).clip(lower=1e-6))
        df["landcover_majority"] = df["landcover_majority"].astype("category")
        df["lisa_label"] = df["lisa_label"].astype("category")
        formula = (f"{y_col} ~ km_closer + is_coastal + log_pop + log_road + C(landcover_majority, Treatment(reference=11)) + C(lisa_label, Treatment(reference='Not signif'))")
        try:
            m = smf.ols(formula, data=df).fit(cov_type="HC1")
        except Exception:
            continue
        rows.append({"outcome": y_col,"coef_km_closer_per100km": float(m.params["km_closer"]),"p_value": float(m.pvalues["km_closer"]),"r2": float(m.rsquared)})
    res = pd.DataFrame(rows).sort_values("p_value")
    results_by_border[d] = res
    sig = res[(res["coef_km_closer_per100km"] > 0) & (res["p_value"] < alpha)].copy()
    sig_by_border[d] = sig
for d in dist_cols:
    print(f"Positive km_closer effects only (per 100 km closer), p < {alpha} | {d}")
    display(sig_by_border[d] if not sig_by_border[d].empty else pd.DataFrame(columns=["outcome","coef_km_closer_per100km","p_value","r2"]))

Positive km_closer effects only (per 100 km closer), p < 0.01 | dist_national_borders_102023_km


,outcome,coef_km_closer_per100km,p_value,r2
20,count_headquarters_or_base_established,0.003927,0.000004,0.015546
22,count_mob_violence,0.129235,0.000013,0.165336
21,count_looting_property_destruction,0.121507,0.002621,0.051851


Positive km_closer effects only (per 100 km closer), p < 0.01 | dist_colonial_borders_102023_km


,outcome,coef_km_closer_per100km,p_value,r2
22,count_mob_violence,0.097886,3.620871e-08,0.165551
20,count_headquarters_or_base_established,0.002774,1.425296e-06,0.015575
24,count_non_violent_transfer_of_territory,0.008695,7.652752e-05,0.025344
8,count_abduction_forced_disappearance,0.061752,2.220361e-03,0.070746
9,count_agreement,0.008180,3.725951e-03,0.021797
6,count_violence_against_civilians,0.181580,4.561163e-03,0.123728
12,count_arrests,0.008251,9.707750e-03,0.049222


Positive km_closer effects only (per 100 km closer), p < 0.01 | dist_religious_borders_102023_km


,outcome,coef_km_closer_per100km,p_value,r2
34,fatalities_total,1.531082,1.616611e-46,0.096599
13,count_attack,0.182536,2.987353e-39,0.133533
6,count_violence_against_civilians,0.244141,2.316464e-38,0.127639
5,count_strategic_developments,0.118323,1.499750e-23,0.076880
8,count_abduction_forced_disappearance,0.055025,1.493347e-22,0.072837
25,count_other,0.021506,1.983101e-21,0.053660
1,count_battles,0.245898,8.325106e-19,0.047128
11,count_armed_clash,0.239616,1.507182e-18,0.046900
29,count_sexual_violence,0.006580,2.938189e-17,0.036620
21,count_looting_property_destruction,0.062234,3.076715e-14,0.053731


Positive km_closer effects only (per 100 km closer), p < 0.01 | dist_ethnic_borders_102023_km


,outcome,coef_km_closer_per100km,p_value,r2
22,count_mob_violence,0.219426,4.056126e-07,0.165808
12,count_arrests,0.029381,2.425476e-05,0.049573
21,count_looting_property_destruction,0.149952,2.815901e-04,0.051742
8,count_abduction_forced_disappearance,0.109212,1.250317e-03,0.070657
